In [0]:
import numpy as np
import itertools
import gym
import matplotlib.pyplot as plt
import matplotlib as mpl


*Please note that for all the following problems, 0 will be high effort and 1 will be low effort.*

# Problem 1

## 1.1

Below we numerically compute the value function of each deterministic policy, and then these are ranked based on the value at the initial state.  The optimal policy is thus 1,1,1,0,0 or low, low, low, high, high.  Even in some of the other policies that are ranked highly, it is mostly playing low effort when closest to -3 (your opponent is closer to winning) and high effort when closest to 3 (you are closer to winning).  This is probably because when you are closer to -3, there is more chance of losing even if you put in the high effort, so the cost of high effort ends up being larger then the expected value from an unlikely win if you spend a high amount of energy.  Therefore, spend low energy and cut costs.  However, when closer to 3, you are more likely to win, and thus it becomes more valuable to put in high effort to increase the chances of the win even more.  

In [0]:
actions = [0, 1]
policies = [p for p in itertools.product(actions, repeat=5)]

d = 3
states = np.arange(-d,d + 1)
probs = [.55,.45]
costs = [-50, -10]
R = 1000

# initialise state values
state_values = np.zeros(len(states))
state_values[d*2] = 1000
theta = 1e-6

allvalues = []
initstatevalues = []

for pol in policies:
    while True:
        delta = 0.0

        for s in range(1,len(state_values)-1):
            new_value = probs[pol[s-1]]*state_values[s+1] + (1-probs[pol[s-1]])*state_values[s-1] + costs[pol[s-1]]
            delta = np.maximum(delta, np.abs(state_values[s] - new_value))
            state_values[s] = new_value

        if delta < theta:
            break

    allvalues.append(np.copy(state_values)) 
    initstatevalues.append(np.copy(state_values[3]))



In [3]:
rankedorder = np.flipud(np.argsort(initstatevalues))
for i in range(len(rankedorder)):
    print(str(i + 1) + ". Initial State Value: " + str(initstatevalues[rankedorder[i]]))
    print("    Policy: " + str(policies[rankedorder[i]] )  )
    print("    All Values: " + str(allvalues[rankedorder[i]]))

1. Initial State Value: 281.6528907652884
    Policy: (1, 1, 1, 0, 0)
    All Values: [   0.           56.52453139  147.83229276  281.65289077  467.43362221
  710.34512999 1000.        ]
2. Initial State Value: 277.1650427015227
    Policy: (1, 1, 1, 1, 0)
    All Values: [   0.           55.31683793  145.14852965  277.1650427   460.74078161
  707.33335173 1000.        ]
3. Initial State Value: 269.2991861176355
    Policy: (1, 1, 1, 0, 1)
    All Values: [   0.           53.20011395  140.44469682  269.29918612  449.0102279
  686.95562535 1000.        ]
4. Initial State Value: 266.2135950190735
    Policy: (1, 1, 1, 1, 1)
    All Values: [   0.           52.3697722   138.5994929   266.21359502  444.40860792
  684.42473435 1000.        ]
5. Initial State Value: 261.6528905420873
    Policy: (1, 1, 0, 0, 0)
    All Values: [   0.           51.14247145  135.87215965  261.65289054  455.47348916
  704.96307012 1000.        ]
6. Initial State Value: 253.15737810536825
    Policy: (1, 1, 0, 1

## 1.2

The action values are shown below, where each pair of values represents the values at a state, the first (index 0) being for taking a high effort action and the second (index 1) being for taking a low effort action.  Based on this, the target policy does not appear optimal because at all non-terminal states except for when you are 2 wins ahead, they are playing high effort, and when 2 wins ahead, they are playing low effot.  However, even just at the first state (s = -2) the value of low effort (112.74) is higher than the value of high effort (56.73) which is what we take given the policy.  Something similar happens at the fourth state (s = 1).  

In [5]:
Q_sa = [[0,0],[0,0],[0,0],[0,0],[0,0]]
C_sa = [[0,0],[0,0],[0,0],[0,0],[0,0]]
        


probs = [.55,.45]
costs = [-50, -10]

R = 1000



episodes = 500000







def targetpolicy(s):
    if s <= 1:
        action = 0
    else:
        action = 1
    return action

def behavepolicy():
    return int(np.random.uniform() < .5)

def r_update(sprime, a):
    if sprime == 3 :
        rout = 1000 + costs[a]
    else :
        rout = costs[a]
    return rout

def moving(prevstate, prob):
    tempmove = int(np.random.uniform() < prob)
    if tempmove == 0:
        tempmove = -1
    newstate = prevstate + tempmove
    return newstate



for i in range(episodes):
    G = 0
    W = 1

    sprime = [0]
    aprime = [behavepolicy()]
    rprime = [0]
    
    t = 0
    while True:
        sprime.append(moving(sprime[t], probs[aprime[t]]))
        rprime.append(r_update(sprime[(t+1)], aprime[t]))
        if abs(sprime[t+1]) == 3:
            break
        

        aprime.append(behavepolicy())
        t += 1


    step = len(sprime)-2
    while True :
        G += rprime[step+1]
        C_sa[sprime[step]+2][aprime[step]] += W
        Q_sa[sprime[step]+2][aprime[step]] += (W/C_sa[sprime[step]+2][aprime[step]])*(G - Q_sa[sprime[step]+2][aprime[step]])
        targetAct = targetpolicy(sprime[step])
        pi_sa = int(targetAct == aprime[step])
        W = W*(pi_sa/.5)
        if W == 0 or step <= 0:
            break
        step -= 1

print(Q_sa)


[[56.72557451955369, 112.73728546876116], [194.82414446706645, 150.01202307349183], [352.15796769137273, 288.24459868149296], [534.5021681991215, 562.1378879036548], [820.3342920106122, 788.4210841128363]]


## 1.3

The optimal policy computed using the off-policy Monte Carlo algorithm using a behavior policy that in each state selects either action equiprobably is shown below, along with the values.  This is 1,1,0,1,0 or low, low, high, low, high.  This is not the top ranked policy in part 1.1, but it is near the top and ranked #6.  While the optimal policy based on this algorithm might not match 1.1 exactly, it should be expected that it is not near the bottom, and that there is some agreement that the policy is good.  This seems to be the case.  This still holds that it's mostly low effort when close to losing, and mostly higher effort when closer to winning, although it does have the person playing low effort at state 1.  

In [9]:
Q_sa = [[0,0],[0,0],[0,0],[0,0],[0,0]]
C_sa = [[0,0],[0,0],[0,0],[0,0],[0,0]]
        

probs = [.55,.45]
costs = [-50, -10]

R = 1000

episodes = 500000




pi_s = np.argmax(Q_sa, axis=1)



def behavepolicy():
    return int(np.random.uniform() < .5)

def r_update(sprime, a):
    if sprime == 3 :
        rout = 1000 + costs[a]
    else :
        rout = costs[a]
    return rout


def moving(prevstate, prob):
    tempmove = int(np.random.uniform() < prob)
    if tempmove == 0:
        tempmove = -1
    newstate = prevstate + tempmove
    return newstate



for i in range(episodes):
    G = 0
    W = 1

    sprime = [0]
    aprime = [behavepolicy()]
    rprime = [0]
    
    t = 0
    while True:
        sprime.append(moving(sprime[t], probs[aprime[t]]))
        rprime.append(r_update(sprime[(t+1)], aprime[t]))
        if abs(sprime[t+1]) == 3:
            break
        
        aprime.append(behavepolicy())
        t += 1

    step = len(sprime)-2
    while True :
        G += rprime[step+1]
        C_sa[sprime[step]+2][aprime[step]] += W
        Q_sa[sprime[step]+2][aprime[step]] += (W/C_sa[sprime[step]+2][aprime[step]])*(G - Q_sa[sprime[step]+2][aprime[step]])
        pi_s[sprime[step]+2] = np.argmax(Q_sa[sprime[step]+2])
        if aprime[step] != pi_s[sprime[step]+2] or step <= 0:
            break
        W = W*(1/.5)
        step -= 1

print(Q_sa)
print(pi_s)

[[30.91307786325384, 36.54735983760822], [122.22101271793751, 163.2379935676167], [328.5416506785813, 323.74409497947244], [522.835876273051, 558.4393621519083], [783.7077636283753, 749.8518856333816]]
[1 1 0 1 0]


# Problem 2
## 2.1

The Q-learning algorithm using an $\epsilon$-greedy policy with $\epsilon = 0.1$ is implemented below to solve the problem.   The values are shown below, and the optimal policy that corresponds to these is 1,1,1,1,0 or low, low, low, low, high which is ranked #2 in part 1.1.  Again, this makes sense that they would both agree that a particular policy is one of the better policies, but not necessarily agree exactly the very optimal policy.  It also makes sense with the reasoning described in 1.1 of using low effort when close to losing and high effort when close to winning.  

In [5]:
probs = [.55,.45]
costs = [-50, -10]
R = 1000

Q_sa = [[0,0],[0,0],[0,0],[0,0],[0,0]]


episodes = 500000
eta = 0.1
epsilon = 0.1

def moving(prevstate, prob):
    tempmove = int(np.random.uniform() < prob)
    if tempmove == 0:
        tempmove = -1
    newstate = prevstate + tempmove
    return newstate

def r_update(sprime, a):
    if sprime == 3 :
        rout = 1000 + costs[a]
    else :
        rout = costs[a]
    return rout

def Q_update(sprime):
    if abs(sprime) == 3 :
        Qout = 0
    else:
        Qout = max(Q_sa[sprime+2])
    return Qout
    

def greedy(Q_s, epsilon):
    if np.random.uniform() > epsilon:
        achoice = np.argmax(Q_s)
    else:
        achoice = int(len(Q_s)*np.random.uniform())
    return achoice
    
    
for i in range(episodes):
    s = 0
    while True:
        a = greedy(Q_sa[s], epsilon)
        sprime = moving(s, probs[a])
        r = r_update(sprime, a)
        Q_sa[s+2][a] += eta*(r + Q_update(sprime) - Q_sa[s+2][a])
        s = sprime
        if abs(s) == 3:
            break
            
print(Q_sa)

[[105.71003390081543, 161.86976617685204], [289.4405522205924, 308.5391487633952], [455.90646004869, 524.383994850877], [648.8936616141898, 666.8805812306487], [804.9377275099197, 771.1552702557246]]


## 2.2

The Sarsa algorithm using an $\epsilon$-greedy policy with $\epsilon = 0.1$ is implemented below to solve the problem.   The values are shown below, and the optimal policy that corresponds to these is 1,1,1,1,0 or low, low, low, low, high which is ranked #2 in part 1.1 and the same as 2.1.  Again, this makes sense that they would both agree that a particular policy is one of the better policies, but not necessarily agree exactly the very optimal policy.  It also makes sense with the reasoning described in 1.1 of using low effort when close to losing and high effort when close to winning.  

In [7]:
probs = [.55,.45]
costs = [-50, -10]
R = 1000

Q_sa = [[0,0],[0,0],[0,0],[0,0],[0,0]]


episodes = 500000
eta = 0.1
epsilon = 0.1


def moving(prevstate, prob):
    tempmove = int(np.random.uniform() < prob)
    if tempmove == 0:
        tempmove = -1
    newstate = prevstate + tempmove
    return newstate

def r_update(sprime, a):
    if sprime == 3 :
        rout = 1000 + costs[a]
    else :
        rout = costs[a]
    return rout

def Q_update(sprime, aprime):
    if abs(sprime) == 3 :
        return 0
    else:
        return Q_sa[sprime+2][aprime]

def greedy(Q_s, epsilon):
    if np.random.uniform() > epsilon:
        achoice = np.argmax(Q_s)
    else:
        achoice = int(len(Q_s)*np.random.uniform())
    return achoice
    
for i in range(episodes):
    s = 0
    a = greedy(Q_sa[s], epsilon)
    while True:
        sprime = moving(s, probs[a])
        r = r_update(sprime, a)
        aprime = greedy(Q_sa[sprime], epsilon)
        Q_sa[s+2][a] += eta*(r + Q_update(sprime, aprime) - Q_sa[s+2][a])
        s = sprime
        a = aprime
        if abs(s) == 3:
            break
        
        
print(Q_sa)

[[-38.051204373148366, -2.2760077168270922], [-19.915888120452923, 64.6000606553439], [121.6879963611231, 153.22251289523803], [353.2439199282871, 419.2019532049044], [768.7397224017661, 676.9025264709442]]


## 2.3

Using the online TD($\lambda$)-algorithm, the values of a random policy are shown below.  While the values are not as high as the top ranked deterministic policy from 1.1, they are still on par with the values of a lot of the other deterministic policies which were evaluated.  

In [10]:
probs = [.55,.45]
costs = [-50, -10]

V = [0,0,0,0,0]
e = [0,0,0,0,0]
episodes = 500000
lamb = 0.9
eta = 0.0001

R = 1000

def r_update(sprime, a):
    if sprime == 3 :
        rout = 1000 + costs[a]
    else :
        rout = costs[a]
    return rout

def moving(prevstate, prob):
    tempmove = int(np.random.uniform() < prob)
    if tempmove == 0:
        tempmove = -1
    newstate = prevstate + tempmove
    return newstate


def deltacalc(r, sprime, s):
    if abs(sprime) == 3 :
        delta = r - V[s+2]
#     elif sprime == 3:
#         delta = r + 1000 - V[s+2]
    else:
        delta = r + V[sprime+2] - V[s+2]
    return delta

for i in range(episodes):
    s = 0
    while True:
        a = int(np.random.uniform() > .5)
        sprime = moving(s, probs[a])
        r = r_update(sprime, a)
        
        delta = deltacalc(r, sprime, s)
        e[s+2] += 1
        for j in range(len(V)):
            V[j] += eta*e[j]*delta
            e[j] = lamb*e[j]
        s = sprime
        if abs(s) == 3:
            break
            
print(V)

[10.342868347566643, 92.82976953549976, 234.46380568023812, 433.8881578560655, 690.5138976664559]


# Problem 3

The problem is solved using the SARSA($\lambda$) algorithm using an $\epsilon$-greedy policy with $\epsilon = 0.1$.  The estimated action values for the different states and policy are shown below where each block of 11 rows represents states with differing numbers of wins (-2 through 2 in order), each row within that represents a different energy level state (0 through 10), and each column is the action taken (0 being high or 1 being low).  In almost every case with a high enough energy (all but the first row of each section), the high effort has a higher value.  If the energy is too low to use high energy, then the value of high energy is 0 as expected, since high energy cannot be used.  As the number of wins goes up and the energy level increases, the values in general increase for both actions as expected since the chances of winning increase.  

In [8]:
probs = [.55,.45]
costs = 1
B = 10
R = 1000
a = 2
p = .2

Q_sa = np.array([[[0.0]*2]*11]*5) 
e_sa = np.array([[[0.0]*2]*11]*5) 



episodes = 500000
eta = 0.001
lamb = 0.9
gamma = 0.9
epsilon = 0.1


def Q_update(sprime, ce, aprime):
    if sprime == 3 :
        Qout= 1000
    elif sprime == -3:
        Qout= 0
    else:
        Qout= Q_sa[sprime+2][ce][aprime]
    return Qout

def moving(prevstate, prob):
    tempmove = int(np.random.uniform() < prob)
    if tempmove == 0:
        tempmove = -1
    newstate = prevstate + tempmove
    return newstate

def r_update(newstate, Action):
    reward = 0
    if newstate == 3:
        reward = R
    return reward

def greedy(Q_s, epsilon):
    if np.random.uniform() > epsilon:
        achoice = np.argmax(Q_s)
    else:
        achoice = int(len(Q_s)*np.random.uniform())
    return achoice

def updateEnergy(ce, a):
    if a == 0:
        ce -= costs
        ce = max(0,ce)
    if np.random.uniform() < p:
        ce += a
        ce = min(10,ce)
    return ce
    

for i in range(episodes):
    if (i+1) % 10000 == 0:
        print("Episode: " + str(i+1))
    s = 0
    ce = B
    a = int(np.random.uniform() > .5)
    while True:
        ceprime = updateEnergy(ce, a)
        sprime = moving(s, probs[a])
        r = r_update(sprime, a)
        if abs(sprime) < 3:
            if ceprime < costs:
                aprime = 1
            else:
                aprime = greedy(Q_sa[sprime + 2][ceprime], epsilon)
        delta = gamma*Q_update(sprime,ceprime, aprime) - Q_sa[s + 2][ce][a]
        e_sa[s+2][ce][a] += 1
        for j in range(len(Q_sa)):
            for k in range(len(Q_sa[0])):
                for l in range(len(Q_sa[0][0])):
                    Q_sa[j][k][l] += eta*e_sa[j][k][l]*delta
                    e_sa[j][k][l] = gamma*lamb*e_sa[j][k][l]
        s = sprime
        a = aprime
        ce = ceprime
        if abs(sprime) == 3: 
            break
        
print(Q_sa)
                


Episode: 10000
Episode: 20000
Episode: 30000
Episode: 40000
Episode: 50000
Episode: 60000
Episode: 70000
Episode: 80000
Episode: 90000
Episode: 100000
Episode: 110000
Episode: 120000
Episode: 130000
Episode: 140000
Episode: 150000
Episode: 160000
Episode: 170000
Episode: 180000
Episode: 190000
Episode: 200000
Episode: 210000
Episode: 220000
Episode: 230000
Episode: 240000
Episode: 250000
Episode: 260000
Episode: 270000
Episode: 280000
Episode: 290000
Episode: 300000
Episode: 310000
Episode: 320000
Episode: 330000
Episode: 340000
Episode: 350000
Episode: 360000
Episode: 370000
Episode: 380000
Episode: 390000
Episode: 400000
Episode: 410000
Episode: 420000
Episode: 430000
Episode: 440000
Episode: 450000
Episode: 460000
Episode: 470000
Episode: 480000
Episode: 490000
Episode: 500000
[[[  0.          50.98780981]
  [ 40.32924522  45.23115147]
  [ 49.36157636  46.08765032]
  [ 35.52594767  59.5492578 ]
  [ 64.72126303  37.22770197]
  [ 68.61550571  45.57748227]
  [ 73.13548048  48.74961866]

# Problem 4

$$P(Y\ loses\ 1\ coin) = P(X\ loses\ 1\ coin) = 0.5$$

Each round in assumed to be independent, so:

$$P(Round\ 1\ Outcome\ \cap \ Round\ 2\ Outcome) =  P(Round\ 1\ Outcome)\times P(Round\ 2\ Outcome)$$

## 4.1

The winning probability for player X given that $x = 1$ is $0.5^y$.  The derivation can be seen below:  


$$P(X\ wins) = P(Y\ loses\ all\ y\ coins) = P(Y\ loses\ round\ 1)\times P(Y\ loses\ round\ 2)\times\ ...\ \times P(Y\ loses\ round\ y) = \prod^y_{i = 1}0.5 = 0.5^y$$




## 4.2

The winning probability for player X given that $y = 1$ is $1 - 0.5^x$.  The derivation can be seen below:  

$$P(X\ wins) = 1 - P(Y\ wins) = 1- P(X\ loses\ all\ x\ coins) = 1- 0.5^x$$


## 4.3 

The winning probability for player X for each value of x and y is given by:
 $$P(X\ wins) = \sum _{i = 0}^{x-1} { y - 1 + i \choose i } \times (0.5) ^ {(y + i)}$$
 
 The derivation can be seen below:  

<br>

$$P(X\ wins) = P(Y\ loses\ (y - 1)\ coins; X\ loses \leq (x-1)\ coins; Y\ loses\ last\ coin\ on\ last\ round)$$ \\



$$P(X\ wins) = P(Y\ loses\ (y - 1); X\ loses\ 0; Y\ loses\ last) + P(Y\ loses\ (y - 1); X\ loses\ 1; Y\ loses\ last) + P(Y\ loses\ (y - 1); X\ loses\ 2; Y\ loses\ last)\ +\ ...\ +\ P(Y\ loses\ (y - 1); X\ loses\ (x - 1); Y\ loses\ last)$$ \
<br>

$$P(X\ wins) = (.5)^{(y-1)}{y-1  \choose 0}(.5) + (.5)^{(y - 1 + 1)}{y -1 + 1 \choose 1}(.5) + (.5)^{(y - 1 + 2)}{y -1 + 2 \choose 2}(.5)\ +\ ...\ +\ .5^{(y - 1 + x - 1)}{y-1 + x-1 \choose x-1 }(.5) $$ \\

<br>

 $$P(X\ wins) = \sum _{i = 0}^{x-1} { y - 1 + i \choose i } \times (0.5) ^ {(y + i)}$$



